# HDP-HMM for changepoint detection in RTT timeseries

In this notebook we benchmark the HDP-HMM and several classical changepoint detection methods on the dataset introduced by Shao et al. [1].

[1] W. Shao, J. Rougier, A. Paris, F. Devienne and M. Viste, "One-to-One Matching of RTT and Path Changes," 2017 29th International Teletraffic Congress (ITC 29), Genoa, 2017, pp. 196-204. https://arxiv.org/pdf/1709.04819.pdf.

In [ ]:
using CSV
using DataFrames
using Glob
using Impute
using PyPlot
using ProgressMeter
using Random
using Statistics
using StatsBase

In [ ]:
using Shao17

In [ ]:
include("../../ParsimoniousMonitoring/notebooks/thesis.jl")

In [ ]:
function load_trace(path; fillmissing=false, nomissingcp=false)
    trace = CSV.read(path, copycols=true)
    
    allowmissing!(trace, :rtt)
    trace[trace.rtt .<= 0.0, :rtt] .= missing

    # Remove change points on missing data points,
    # and one time step after.
    if nomissingcp
        trace[ismissing.(trace.rtt), :cp] .= 0
        trace[circshift(ismissing.(trace.rtt), 1), :cp] .= 0
    end
    
    if fillmissing
        Impute.locf!(trace.rtt)
        Impute.nocb!(trace.rtt)
        disallowmissing!(trace, :rtt)
    end
    
    trace
end

In [ ]:
function benchmark(f, trace; window = 2)
    Random.seed!(2020)
    rtt = collect(trace.rtt)

    # Julia and R arrays are 1-indexed, while Python lists are 0-indexed,
    # so we subtract 1 to every indices.
    fact = findall(trace.cp .== 1) .- 1
    detection = f(rtt) .- 1

    # Like in Shao paper, we replace missing values with -3.0 when
    # weighting RTT changes.
    evaluation_window_weighted(coalesce.(rtt, -3.0), fact, detection, window=window)
end

In [ ]:
function benchmarkall(methods, files; trace_args=Dict())
    results = DataFrame(
        [String[], String[], Float64[], Float64[], Float64[], Float64[], Float64[]],
        [:file, :method, :precision, :recall, :recall_w, :f2, :f2_w]
    )

    @showprogress for file in files
        trace = load_trace(file; trace_args...)
        for (name, method) in methods
            eval = benchmark(method, trace)
            push!(results, (file, name, eval.precision, eval.recall, eval.score, f2(eval), f2w(eval)))
        end
    end
    
    results
end

In [ ]:
function plot_results(results)
    methods = unique(results.method)

    metrics = [:precision, :recall, :recall_w, :f2, :f2_w]
    names = ["Precision", "Recall", "Recall (weighted)", "F2", "F2 (weighted)"]
    kwargs = Dict(
        "cpt_np&MBIC" => Dict(:color => "gray", :linestyle => "--"),
        "cpt_poisson&MBIC" => Dict(:color => "purple", :linestyle => "--"),
        "HDP-HMM" => Dict(:color => "red", :label => "HDPHMM"),
        "DPMM" => Dict(:color => "pink", :label => "DPMM"),
    )

    fig, axes = subplots(ncols=length(metrics), figsize=(20, 3), sharey=true)
    # TODO: Reduce space between boxes

    for (ax, metric, name) in zip(axes, metrics, names)
        for method in methods
            df = results[results.method .== method, metric]
            supp = 0:0.001:1
            label = replace(method, "_" => "\\_")
            label = replace(label, "&" => "\\&")
            ax.plot(supp, ecdf(df)(supp); label = label, kwargs[method]...)
        end
        ax.set_xlabel(name)
    end
    
    axes[1].set_ylabel("CDF")
    axes[end].legend(loc="upper left", fontsize=12)
end

## Main
### Dataset

In [ ]:
files = glob("*.csv", "../external/rtt/dataset/real_trace_labelled/");

In [ ]:
# Average percentage of missing data
mean(map(files) do file
    trace = load_trace(file)
    sum(ismissing.(trace.rtt)) / length(trace.rtt) * 100
end)

### Benchmark

In [ ]:
enablemissing(1.0)

In [ ]:
methods = Dict(
    # Like in Shao paper, we replace missing values with 1e3.
    "cpt_np&MBIC"      => rtt -> cpt_np(coalesce.(rtt, 1000.0), "MBIC", 3),
    "cpt_poisson&MBIC" => rtt -> cpt_poisson(coalesce.(rtt, 1000.0), "MBIC", 3),
    "HDP-HMM"          => rtt -> cpt_hmm(coalesce.(rtt, 1000.0)),
    "DPMM"             => rtt -> cpt_mm(coalesce.(rtt, 1000.0)),
);

In [ ]:
# results_paper = benchmarkall(methods, files, trace_args=Dict(:fillmissing => false, :nomissingcp => false));
results_paper = CSV.read("../results/cpt_real_paper.csv");

In [ ]:
plot_results(results_paper)

In [ ]:
metrics = [:precision, :recall, :recall_w, :f2, :f2_w]
combine(groupby(results_paper, :method), [metric => median for metric in metrics])

In [ ]:
# results_fixed = benchmarkall(methods, files, trace_args=Dict(:fillmissing => true, :nomissingcp => true));
results_fixed = CSV.read("../results/cpt_real_fixed.csv");

In [ ]:
plot_results(results_fixed)

In [ ]:
metrics = [:precision, :recall, :recall_w, :f2, :f2_w]
combine(groupby(results_fixed, :method), [metric => median for metric in metrics])

In [ ]:
# CSV.write("../results/cpt_real_paper.csv", results_paper);
# CSV.write("../results/cpt_real_fixed.csv", results_fixed);